In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("mai_clients.csv")
df.Datetime = df.Datetime.apply(pd.to_datetime)
serivce_df = (
    df.groupby(by=["ShopId", "EmployeeId"])
    .Datetime.apply(lambda x: x[1:] - x.shift(1).dropna())
    .reset_index(drop=True)
)
df_time = pd.merge(df, serivce_df, how="left", left_index=True, right_index=True)
df_time = df_time.rename(columns={"Datetime_y": "ServiceTime"}, inplace=False)
df_time.ServiceTime = df_time.ServiceTime.apply(lambda x: x.total_seconds())
emp_stats = df_time.groupby(by=["ShopId", "EmployeeId"]).aggregate("median")
Shops_columns = {"Total_money"}
emp_criterions = set(emp_stats.columns).difference(Shops_columns)
emp_criterions = list(emp_criterions)


In [4]:
data = pd.read_csv("ranked.csv").set_index("EmployeeId")
emp_stats_df = pd.merge(
    emp_stats.reset_index(), data, left_on=["EmployeeId"], right_index=True
).set_index(["ShopId", "EmployeeId"])

emp_stats_df

Total  employee_total_money  Total_money  ServiceTime  \
ShopId EmployeeId                                                            
4      4           444.000                444.00     21785.55         26.0   
5      4             5.050                481.36     31872.42        434.5   
4      15            2.550                 28.75     21785.55         56.0   
5      15            6.250                391.11     31872.42        357.0   
4      18            2.675                114.05     21785.55         62.0   
5      18            5.350               2202.83     31872.42        430.0   
4      21            2.800                505.00     21785.55         79.0   
       28            2.600               2118.19     21785.55        113.0   
5      28            4.825                772.46     31872.42        517.0   
4      30            2.575                589.84     21785.55         77.0   
       98            2.950                 38.50     21785.55         51.0   
       161           3.200                 25.00     21785.55        135.0   
5      161           4.800               3202.64     31872.42        413.0   
4      173           2.750               5569.18     21785.55        102.0   
       176           2.750               2287.68     21785.55         98.0   
5      176           4.800               2785.21     31872.42        347.5   
4      192           4.300                 22.45     21785.55         69.0   
5      192           5.800               1687.75     31872.42        483.0   
4      196           2.900               2870.65     21785.55         85.0   
5      196           5.000               1107.91     31872.42        407.0   
4      205           2.750               7105.46     21785.55         96.0   
       209           2.825                 66.80     21785.55        206.5   
5      209           5.300               3432.21     31872.42        476.0   
       7             7.350               1466.45     31872.42        507.0   
       10            5.600               3643.76     31872.42        416.0   
       13            6.975               1858.13     31872.42        446.0   
       14            4.900               3609.42     31872.42        456.0   
       29            4.500                332.97     31872.42        428.5   
       70            7.100                388.21     31872.42        664.5   
       100           8.750                 62.60     31872.42        701.5   
       144           5.000               2458.54     31872.42        381.0   
       187           4.300                465.98     31872.42        491.0   
       206           5.150               1208.30     31872.42        412.0   
       207           5.950                314.58     31872.42        310.0   

                   rank employee_total_money  rank Total  rank ServiceTime  
ShopId EmployeeId                                                           
4      4                                16.0         1.0              15.0  
5      4                                16.0         1.0              15.0  
4      15                               20.0         3.0               9.0  
5      15                               20.0         3.0               9.0  
4      18                               11.0         6.0              17.0  
5      18                               11.0         6.0              17.0  
4      21                               18.0        22.0               3.0  
       28                                9.0        10.0              21.0  
5      28                                9.0        10.0              21.0  
4      30                               17.0        24.0               2.0  
       98                               25.0        21.0               1.0  
       161                               8.0         7.0              19.0  
5      161                               8.0         7.0              19.0  
4      173                               2.0        23.0 

In [5]:
# order = [Total	empol_total_money	ServiceTime]
D = np.array(
    [[1, 7, 3],
     [1/7, 1, 1/7],
     [1/3, 7, 1]]
)


def EM(D):
    n = D.shape[0]
    def MRCI(n):
        table1 = [0, 0, 0.52, 0.89, 1.11, 1.25, 1.35, 1.40, 1.45, 1.49, 1.52, 1.54, 1.56, 1.58, 1.59]
        if n >= len(table1):
            raise ValueError("n too chonky")
        return table1[n-1]

    def get_CR(max_lambda):
        return ((max_lambda - n) / (n - 1)) / MRCI(n)

    w,v = np.linalg.eig(D)
    lambda_max = np.max(w).astype(np.float32)
    weights = v[:,np.argmax(w)].astype(np.float32)
    return {"w": weights, "CR": get_CR(lambda_max)}


em_res = EM(D)
local_weights = em_res["w"]
local_weights, em_res["CR"]

<ipython-input-5-d03c14de570e>:19: ComplexWarning: Casting complex values to real discards the imaginary part
  lambda_max = np.max(w).astype(np.float32)
<ipython-input-5-d03c14de570e>:20: ComplexWarning: Casting complex values to real discards the imaginary part
  weights = v[:,np.argmax(w)].astype(np.float32)


(array([0.89768934, 0.08891758, 0.43156403], dtype=float32),
 0.1303950181374183)

In [7]:

def get_emp_weights(data, local_weights, aggregation_method, order=True):

    data_inv = data.applymap(lambda x: 1.0 / x) if order else data
    data_inv_normalized = data_inv.divide(data_inv.sum(axis=1), axis=0)

    if aggregation_method == "distributive":

        data_inv_normalized_weighted = data_inv_normalized.multiply(
            local_weights, axis=1
        )
        result = data_inv_normalized_weighted.sum(axis=1)
        result_norm = (result / result.sum()).sort_values(ascending=False)


    elif aggregation_method == "multi":

        data_inv_normalized_weighted = data_inv_normalized.pow(
            local_weights, axis=1
        )
        result = data_inv_normalized_weighted.prod(axis=1)
        result_norm = (result / result.sum()).sort_values(ascending=False)

    elif aggregation_method == "ideal_point":

        data_inv_normalized_weighted = data_inv_normalized.divide(
            data_inv_normalized.max(axis=1), axis=0
        )
        result = data_inv_normalized_weighted.sum(axis=1)
        result_norm = (result / result.sum()).sort_values(ascending=False)

    return result_norm

emp_df = pd.concat([pd.Series(get_emp_weights(data, local_weights, method).index) \
           for method in ('distributive', 'ideal_point', 'multi')], axis=1)
emp_df.columns = ('distributive', 'ideal_point', 'multi')
emp_df[['distributive']]

,distributive
0,205
1,173
2,176
3,14
4,10
5,196
6,144
7,28
8,13
9,187


In [13]:

temp = (
    emp_stats_df.reset_index()
    .groupby("EmployeeId")
    .apply(
        lambda x: x[
            (list(emp_criterions))
            + ["rank Total", "rank employee_total_money", "rank ServiceTime"]
        ].sort_values("employee_total_money", ascending=False)
    )
)

pd.merge(
    temp.reset_index(),
    emp_df[["distributive"]].reset_index(),
    left_on="EmployeeId",
    right_on="distributive",
).sort_values("index", ascending=True)


,EmployeeId,level_1,ServiceTime,employee_total_money,Total,rank Total,rank employee_total_money,rank ServiceTime,index,distributive
29,205,20,96.0,7105.46,2.750,23.0,1.0,4.0,0,205
21,173,13,102.0,5569.18,2.750,23.0,2.0,5.0,1,173
22,176,15,347.5,2785.21,4.800,9.0,3.0,12.0,2,176
23,176,14,98.0,2287.68,2.750,9.0,3.0,12.0,2,176
5,14,26,456.0,3609.42,4.900,18.0,6.0,14.0,3,14
3,10,24,416.0,3643.76,5.600,15.0,5.0,10.0,4,10
28,196,19,407.0,1107.91,5.000,8.0,4.0,17.0,5,196
27,196,18,85.0,2870.65,2.900,8.0,4.0,17.0,5,196
18,144,30,381.0,2458.54,5.000,17.0,10.0,7.0,6,144
11,28,7,113.0,2118.19,2.600,10.0,9.0,21.0,7,28
